# Map Moment Tensors to Events 
1. Unpickle events, stations, bounding box, and shape-file
1. Read moment tensor CSV file
1. Map (attach) moment tensors to events
1. Plot overlay of moment tenosrs on the Groningen Field 
1. In local coordinates: Plot above overlay of moment tensors

#### 1. Unpickle events, station, bound box, and shape-file

In [ ]:
!pwd

In [ ]:
import pickle
from gnam.events.gevents import gevents as gevents
from gnam.events.gstations import gstations as gstations
from gnam.events.munge.knmi import correct_station_depths as csd_f
import shapefile as sf

dpath = '../../../data_notebooks/'

#Unpickle events
print('Unpickling Events')
f = open(dpath + 'pickled/events.pickle', 'rb')
dill_events = pickle.load(f)
f.close()
    

#Unpickle stations
print('Unpickling Station Traces')
f = open(dpath + 'pickled/straces.pickle', 'rb')
dill_straces = pickle.load(f)
f.close()
# This is kind of hokey, but it works for now.
# Some of the stations depths do not follow the 
# 50, 100, 150, 200 meter depths -- possibly because
# the boreholes are slanted. To correct for this,
# a hard coded "patch/update" is applied. See the
# code for details and update values.
dill_straces.correct_stations(csd_f)


#Unpickle bounding box
print('Unpickling Bounding Box')
f = open(dpath + 'pickled/model_bbox.pickle', 'rb')
gf_bbox = pickle.load(f)
f.close()


#Unpickle groningen shape-file
print('Unpickling Groningen Shape-File')
mysf = sf.Reader(dpath + 'data/GroningenFieldShapeFile/Groningen_field')
    
print('Done')

#### 2. Read moment tensor CSV file

In [ ]:
from gnam.events.mtensors import mtensors
import pandas as pd
import numpy as np

#rdeg = -1*gf_bbox.getRotDeg()
#rdeg = 0
#print('rdeg:',rdeg)

gf_mts = mtensors(dpath + 'data/event_moments.csv')
gf_mts_local = mtensors(dpath + 'data/event_moments.csv',gf_bbox)
gf_df = gf_mts.get_df()
print('Moment Tensor DataFrame:\n',gf_df)
print()

print('Moment Tensors:')
for imt in range(len(gf_mts)):
    print('Tensor[%d]:' %(imt))
    print(gf_mts[imt])

print()
print()
print(gf_mts[0])
print()
print(gf_mts_local[0])
    

#### 3. Map (attach) moment tensors to events

In [ ]:
# get event catalog of the events withing the bounding box
e_cat = dill_events.getIncCatalog()


# This is a bit hokey, but it works. Here we update the
# event time from the moment tensor CSV file with thouse
# from the event catalog
gf_mts.update_utcdatetime(e_cat)


# Create a dictionary that maps moment tensors to events
e2mt_dict = gf_mts.map_events_2_tensors(e_cat)
e2mt_keys = e2mt_dict.keys()

# Print a comparison of events to moment tensors
for key in e2mt_keys:
    print('UTC: event[%d][Date] = %s' %(key,e_cat[key].origins[0].time))
    print('UTC:    MT[%d][Date] = %s' %(key,e2mt_dict[key]['Date']))
    print('Mag: event[%d][Date] = %s' %(key,e_cat[key].magnitudes[0].mag))
    print('Mag:    MT[%d][Date] = %s' %(key,e2mt_dict[key]['ML']))
    print()

#### 4. Plot overlay of moment tenosrs on the Groningen Field 

In [ ]:
#%matplotlib
import matplotlib.pyplot as plt 

#get coordinates for the Shape-File
s = mysf.shape(0)
shape_xy = np.asarray(s.points)

#get beachballs for the moment tensors
aki_bballs = gf_mts.get_aki_beachballs(diam=1700)
cmt_bballs = gf_mts.get_cmt_beachballs(diam=1700,fc='red')

#get coordinates for the moment tensors
aki_xc = gf_mts.get_xcoords()

event_coords = dill_events.getIncCoords()[3:,:] #some events have no inverted MTs
#print('ecoords:\n',event_coords)

diag = np.zeros((2,2))
diag[0,0] = gf_bbox.getCLoop()[0,0] 
diag[1,0] = gf_bbox.getCLoop()[2,0] 
diag[0,1] = gf_bbox.getCLoop()[0,1] 
diag[1,1] = gf_bbox.getCLoop()[2,1] 


#Plot shape-file, bounding box, and Aki Beachballs
fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(shape_xy[:,0],shape_xy[:,1],s=1,c='black',zorder=0)
ax.plot(gf_bbox.getCLoop()[:,0],gf_bbox.getCLoop()[:,1],c='green',zorder=1)
#ax.plot([gf_bbox.getCLoop()[0,0],gf_bbox.getCLoop()[2,0]],[gf_bbox.getCLoop()[0,1],gf_bbox.getCLoop()[2,1]],c='green',zorder=1)
ax.plot(diag[:,0],diag[:,1],c='green',zorder=1)
for bb in aki_bballs: # add the beachballs
    bb.set_zorder(2)
    ax.add_collection(bb)
ax.scatter(event_coords[:,0],event_coords[:,1],s=90,c='red',marker='*',zorder=5)
ax.set_title('Aki BeachBalls, %d Events' %(len(aki_bballs)))

#Plot shape-file, bounding box, and Harvard CMT Beachballs
fig1, ax1 = plt.subplots(1,figsize=(8,8))
ax1.scatter(shape_xy[:,0],shape_xy[:,1],s=1,c='black',zorder=0)
ax1.plot(gf_bbox.getCLoop()[:,0],gf_bbox.getCLoop()[:,1],c='green',zorder=1)
for bb in cmt_bballs:
    bb.set_zorder(2)
    ax1.add_collection(bb)
ax1.scatter(event_coords[:,0],event_coords[:,1],s=100,c='blue',marker='.',zorder=5)
ax1.set_title('Harvard CMT BeachBalls, %d Events' %(len(cmt_bballs)))

plt.show()

#### 5. In local coordinates: Plot overlay of moment tenosrs on the Groningen Field 

In [ ]:
#%matplotlib
import copy
import matplotlib.pyplot as plt 

#get coordinates for the Shape-File
s = mysf.shape(0)
g_shape_xy = np.asarray(s.points)
shape_xy = np.zeros_like(g_shape_xy)

#get beachballs for the moment tensors
aki_bballs = gf_mts_local.get_aki_beachballs(diam=1700,fc='blue',is_local=True)
cmt_bballs = gf_mts_local.get_cmt_beachballs(diam=1700,fc='red',is_local=True)

event_coords = dill_events.getLocalIncCoords()[3:,:] #some events have no inverted MTs
#print('ecoords:\n',event_coords)

#get local coordinates for groningen field and bbox
l_gf_bbox = copy.deepcopy(gf_bbox)
o_bb = gf_bbox.getOrigin()
l_gf_bbox.translate(-o_bb[0],-o_bb[1])
neg_rotdeg = -gf_bbox.getRotDeg()
l_gf_bbox.rotate(neg_rotdeg)

for i in range(len(g_shape_xy)):
    xc = g_shape_xy[i,0]
    yc = g_shape_xy[i,1]
    lxc,lyc = gf_bbox.convert_extern_coords_2_local(xc,yc) 
    shape_xy[i,0] = lxc
    shape_xy[i,1] = lyc

#Plot shape-file, bounding box, and Aki Beachballs
fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(shape_xy[:,0],shape_xy[:,1],s=1,c='black',zorder=0)
ax.plot(l_gf_bbox.getCLoop()[:,0],l_gf_bbox.getCLoop()[:,1],c='green',zorder=1)
for bb in aki_bballs: # add the beachballs
    bb.set_zorder(2)
    ax.add_collection(bb)
ax.scatter(event_coords[:,0],event_coords[:,1],s=90,c='red',marker='*',zorder=5)
ax.set_title('Aki BeachBalls, %d Events' %(len(aki_bballs)))

#Plot shape-file, bounding box, and Harvard CMT Beachballs
fig1, ax1 = plt.subplots(1,figsize=(8,8))
ax1.scatter(shape_xy[:,0],shape_xy[:,1],s=1,c='black',zorder=0)
ax1.plot(l_gf_bbox.getCLoop()[:,0],l_gf_bbox.getCLoop()[:,1],c='green',zorder=1)
for bb in cmt_bballs:
    bb.set_zorder(2)
    ax1.add_collection(bb)
ax1.scatter(event_coords[:,0],event_coords[:,1],s=100,c='blue',marker='.',zorder=5)
ax1.set_title('Harvard CMT BeachBalls, %d Events' %(len(cmt_bballs)))

plt.show()

## Finished